In [25]:
# I want to predict the individual teams total points and add them together so build dataframes with teams offensive stats and opponents defensive stats

# Retrieve data, clean data and store in folder to train model on

In [26]:
# Imports

import nflreadpy as nfl
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

## Retrieve data

In [27]:
schedule = nfl.load_schedules(seasons=2025)
schedule = schedule.to_pandas()

team_stats = nfl.load_team_stats(seasons=2025)
team_stats = team_stats.to_pandas()

In [28]:
team_stats

season  week team season_type opponent_team  completions  attempts  \
0      2025     1  ARI         REG            NO           21        29   
1      2025     1  ATL         REG            TB           27        42   
2      2025     1  BAL         REG           BUF           14        19   
3      2025     1  BUF         REG           BAL           33        46   
4      2025     1  CAR         REG           JAX           18        35   
..      ...   ...  ...         ...           ...          ...       ...   
237    2025     8  PIT         REG            GB           24        36   
238    2025     8   SF         REG           HOU           19        32   
239    2025     8   TB         REG            NO           15        24   
240    2025     8  TEN         REG           IND           22        38   
241    2025     8  WAS         REG            KC           21        30   

     passing_yards  passing_tds  passing_interceptions  ...  pat_made  \
0              163            2                      0  ...         2   
1              298            1                      0  ...         2   
2              209            2                      0  ...         4   
3              394            2                      0  ...         2   
4              154            1                      2  ...         1   
..             ...          ...                    ...  ...       ...   
237            219            2                      0  ...         1   
238            193            2                      1  ...         1   
239            152            0                      0  ...         2   
240            259            1                      1  ...         2   
241            213            1                      2  ...         1   

     pat_att  pat_missed  pat_blocked  pat_pct  gwfg_made  gwfg_att  \
0          2           0            0      1.0          0         0   
1          2           0            0      1.0          0         0   
2          5           1            0      0.8          0         0   
3          2           0            0      1.0          1         1   
4          1           0            0      1.0          0         0   
..       ...         ...          ...      ...        ...       ...   
237        1           0            0      1.0          0         0   
238        1           0            0      1.0          0         0   
239        2           0            0      1.0          0         0   
240        2           0            0      1.0          0         0   
241        1           0            0      1.0          0         0   

     gwfg_missed  gwfg_blocked  gwfg_distance  
0              0             0              0  
1              0             0              0  
2              0             0              0  
3              0             0             32  
4              0             0              0  
..           ...           ...            ...  
237            0             0              0  
238            0             0              0  
239            0             0              0  
240            0             0              0  
241            0             0              0  

[242 rows x 102 columns]

In [29]:
# Filter out data I don't want

cols_to_drop = ['season', 
 'season_type',
 'sack_fumbles',
 'passing_air_yards',
 'passing_yards_after_catch',
 'passing_first_downs',
 'passing_2pt_conversions',
 'rushing_2pt_conversions',
 'receptions',
 'targets',
 'receiving_yards',
 'receiving_tds',
 'receiving_fumbles',
 'receiving_air_yards',
 'receiving_yards_after_catch',
 'receiving_first_downs',
 'receiving_2pt_conversions',
 'special_teams_tds',
 'misc_yards',
 'fumble_recovery_own',
 'fumble_recovery_yards_own',
 'fumble_recovery_opp',
 'fumble_recovery_yards_opp',
 'fumble_recovery_tds',
 'timeouts',
 'fg_made',
 'fg_missed',
 'fg_blocked',
 'fg_long',
 'fg_made_0_19',
 'fg_made_20_29',
 'fg_made_30_39',
 'fg_made_40_49',
 'fg_made_50_59',
 'fg_made_60_',
 'fg_missed_0_19',
 'fg_missed_20_29',
 'fg_missed_30_39',
 'fg_missed_40_49',
 'fg_missed_50_59',
 'fg_missed_60_',
 'fg_made_list',
 'fg_missed_list',
 'fg_blocked_list',
 'fg_made_distance',
 'fg_missed_distance',
 'fg_blocked_distance', 
 'pat_made',
 'pat_att',
 'pat_missed',
 'pat_blocked',
 'gwfg_made',
 'gwfg_att',
 'gwfg_missed',
 'gwfg_blocked',
 'gwfg_distance']

team_stats = team_stats.drop(columns=cols_to_drop)

In [30]:
# for each week group by team sum/avg all previous weeks 
team_stats = team_stats.sort_values(["team", "week"]).reset_index(drop=True)

In [31]:
# Cumulative averages up to a certain week

avg_cols = ['passing_epa',
 'passing_cpoe',
 'rushing_epa',
 'receiving_epa',
 'fg_pct',
 'pat_pct',
 'completions',
 'attempts',
 'passing_yards',
 'passing_tds',
 'passing_interceptions',
 'sacks_suffered',
 'sack_yards_lost',
 'sack_fumbles_lost',
 'carries',
 'rushing_yards',
 'rushing_tds',
 'rushing_fumbles_lost',
 'rushing_first_downs',
 'receiving_fumbles_lost',
 'def_tackles_solo',
 'def_tackles_with_assist',
 'def_tackle_assists',
 'def_tackles_for_loss',
 'def_tackles_for_loss_yards',
 'def_fumbles_forced',
 'def_sacks',
 'def_sack_yards',
 'def_qb_hits',
 'def_interceptions',
 'def_interception_yards',
 'def_pass_defended',
 'def_tds',
 'def_fumbles',
 'def_safeties',
 'penalties',
 'penalty_yards',
 'punt_returns',
 'punt_return_yards',
 'kickoff_returns',
 'kickoff_return_yards',
 'fg_att']

team_stats[[f"avg_{c}" for c in avg_cols]] = (
    team_stats.groupby("team")[avg_cols]
      .expanding()
      .mean()
      .reset_index(level=0, drop=True)
)


In [32]:
team_stats

week team opponent_team  completions  attempts  passing_yards  \
0       1  ARI            NO           21        29            163   
1       2  ARI           CAR           17        25            220   
2       3  ARI            SF           22        35            159   
3       4  ARI           SEA           27        41            200   
4       5  ARI           TEN           23        32            220   
..    ...  ...           ...          ...       ...            ...   
237     4  WAS           ATL           16        27            156   
238     5  WAS           LAC           15        26            231   
239     6  WAS           CHI           19        26            211   
240     7  WAS           DAL           16        32            219   
241     8  WAS            KC           21        30            213   

     passing_tds  passing_interceptions  sacks_suffered  sack_yards_lost  ...  \
0              2                      0               5              -33  ...   
1              1                      1               1               -9  ...   
2              1                      0               1               -5  ...   
3              2                      2               6              -36  ...   
4              0                      0               3              -28  ...   
..           ...                    ...             ...              ...  ...   
237            2                      1               2               -9  ...   
238            1                      0               1               -5  ...   
239            3                      1               3               -6  ...   
240            1                      1               4              -14  ...   
241            1                      2               2              -13  ...   

     avg_def_tds  avg_def_fumbles  avg_def_safeties  avg_penalties  \
0            0.0         0.000000          0.000000       9.000000   
1            0.0         0.000000          0.000000      10.500000   
2            0.0         0.000000          0.333333       8.666667   
3            0.0         0.000000          0.250000       8.250000   
4            0.0         0.200000          0.200000       8.200000   
..           ...              ...               ...            ...   
237          0.0         0.500000          0.000000       7.500000   
238          0.0         0.400000          0.000000       7.400000   
239          0.0         0.333333          0.000000       7.000000   
240          0.0         0.285714          0.142857       7.571429   
241          0.0         0.250000          0.125000       7.000000   

     avg_penalty_yards  avg_punt_returns  avg_punt_return_yards  \
0            54.000000          3.000000              30.000000   
1            75.000000          2.000000              29.500000   
2            60.000000          1.666667              22.333333   
3            56.500000          1.500000              20.500000   
4            54.600000          1.600000              19.600000   
..                 ...               ...                    ...   
237          55.750000          3.000000              44.250000   
238          54.200000          2.400000              35.400000   
239          51.833333          2.166667              31.000000   
240          61.285714          2.000000              27.714286   
241          58.000000          1.875000              26.000000   

     avg_kickoff_returns  avg_kickoff_return_yards  avg_fg_att  
0               3.000000                 73.000000    3.000000  
1               3.500000                 80.000000    2.500000  
2               3.666667                 91.666667    2.666667  
3               4.000000                100.250000    2.500000  
4               4.000000                 98.200000    2.000000  
..                   ...                       ...         ...  
237             3.750000                126.500000    2.500000  
238             3.600000 

## Retrieve Time of Possesion Stats

In [33]:
for attempt in range(1, 5 + 1):
    try:
        # set up the browser
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
        
        # visit T.O.P. Website
        driver.get("https://www.teamrankings.com/nfl/stat/average-time-of-possession-net-of-ot")

        table = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//*[@id='DataTables_Table_0']"))
                    )

        df = pd.read_html(table.get_attribute("outerHTML"))[0]

        driver.quit()
        break

    except Exception as e:
        driver.quit()
        print(f"Attempt {attempt} failed: {e}")
        if attempt == 5:
            print("No retries left — raising.")
            raise

C:\Users\tyler\AppData\Local\Temp\ipykernel_17448\26274015.py:13: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table.get_attribute("outerHTML"))[0]


In [34]:
mapping = {'Buffalo': 'BUF',
 'LA Chargers': 'LAC',
 'Kansas City': 'KC',
 'Houston': 'HOU',
 'Detroit': 'DET',
 'Carolina': 'CAR',
 'Arizona': 'ARI',
 'New England': 'NE',
 'Jacksonville': 'JAX',
 'Chicago': 'CHI',
 'Tampa Bay': 'TB',
 'Atlanta': 'ATL',
 'NY Giants': 'NYG',
 'Denver': 'DEN',
 'Green Bay': 'GB',
 'San Francisco': 'SF',
 'Philadelphia': 'PHI',
 'Cleveland': 'CLE',
 'Dallas': 'DAL',
 'NY Jets': 'NYJ',
 'Indianapolis': 'IND',
 'Las Vegas': 'LV',
 'Minnesota': 'MIN',
 'Seattle': 'SEA',
 'Miami': 'MIA',
 'LA Rams': 'LAR',
 'Baltimore': 'BAL',
 'New Orleans': 'NO',
 'Tennessee': 'TEN',
 'Washington': 'WAS',
 'Pittsburgh': 'PIT',
 'Cincinnati': 'CIN'}

df['team_abb'] = df['Team'].map(mapping)

In [35]:
top_dict = pd.Series(df['2025'].values, index=df['team_abb']).to_dict()

## Add stats for previous weeks in schedule

In [36]:
desired_columns = ['week', 'team', 'opponent_team', 'avg_passing_epa', 'avg_passing_cpoe', 'avg_rushing_epa', 'avg_receiving_epa', 'avg_fg_pct', 'avg_pat_pct', 'avg_completions',
       'avg_attempts', 'avg_passing_yards', 'avg_passing_tds', 'avg_passing_interceptions', 'avg_sacks_suffered', 'avg_sack_yards_lost', 'avg_sack_fumbles_lost', 'avg_carries',
       'avg_rushing_yards', 'avg_rushing_tds', 'avg_rushing_fumbles_lost', 'avg_rushing_first_downs', 'avg_receiving_fumbles_lost', 'avg_def_tackles_solo', 'avg_def_tackles_with_assist',
       'avg_def_tackle_assists', 'avg_def_tackles_for_loss', 'avg_def_tackles_for_loss_yards', 'avg_def_fumbles_forced', 'avg_def_sacks', 'avg_def_sack_yards', 'avg_def_qb_hits',
       'avg_def_interceptions', 'avg_def_interception_yards', 'avg_def_pass_defended', 'avg_def_tds', 'avg_def_fumbles', 'avg_def_safeties', 'avg_penalties', 'avg_penalty_yards',
       'avg_punt_returns', 'avg_punt_return_yards', 'avg_kickoff_returns', 'avg_kickoff_return_yards', 'avg_fg_att']

team_stats = team_stats[desired_columns]

# add 1 week because thats the data we can actually use for the week we predict
team_stats['week'] = team_stats['week'] - 1

C:\Users\tyler\AppData\Local\Temp\ipykernel_17448\2601645564.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_stats['week'] = team_stats['week'] - 1


In [37]:
offensive = ['week', 'team', 'opponent_team', 'avg_passing_epa', 'avg_passing_cpoe', 'avg_rushing_epa', 'avg_receiving_epa', 'avg_fg_pct', 'avg_pat_pct', 'avg_completions',
       'avg_attempts', 'avg_passing_yards', 'avg_passing_tds', 'avg_passing_interceptions', 'avg_sacks_suffered', 'avg_sack_yards_lost', 'avg_sack_fumbles_lost', 'avg_carries',
       'avg_rushing_yards', 'avg_rushing_tds', 'avg_rushing_fumbles_lost', 'avg_rushing_first_downs', 'avg_receiving_fumbles_lost', 'avg_penalties', 'avg_penalty_yards',
       'avg_punt_returns', 'avg_punt_return_yards', 'avg_kickoff_returns', 'avg_kickoff_return_yards', 'avg_fg_att']

defensive = ['week', 'team', 'opponent_team', 'avg_def_tackles_solo', 'avg_def_tackles_with_assist', 'avg_def_tackle_assists', 'avg_def_tackles_for_loss', 'avg_def_tackles_for_loss_yards', 
        'avg_def_fumbles_forced', 'avg_def_sacks', 'avg_def_sack_yards', 'avg_def_qb_hits', 'avg_def_interceptions', 'avg_def_interception_yards', 'avg_def_pass_defended', 
        'avg_def_tds', 'avg_def_fumbles', 'avg_def_safeties']

offensive_stats = team_stats[offensive]
defensive_stats = team_stats[defensive]

In [38]:
# columns from schedule: week, home_team, away_team, home_score, away_score, total_line
schedule = schedule[schedule['week'] > 1]
schedule_interests = schedule[['game_id', 'week', 'home_team', 'home_score', 'away_team', 'away_score']]

In [39]:
schedule_away = schedule_interests
schedule_home = schedule_interests

In [40]:
schedule_away = schedule_away.rename(columns={'home_team':'opponent', 'away_team':'team', 'away_score':'points'})
schedule_away = schedule_away[['game_id', 'week', 'team', 'opponent', 'points']]

schedule_home = schedule_home.rename(columns={'away_team':'opponent', 'home_team':'team', 'home_score':'points'})
schedule_home = schedule_home[['game_id', 'week', 'team', 'opponent', 'points']]

In [41]:
schedule_total = pd.concat([schedule_away, schedule_home])

In [42]:
schedule_total

game_id  week team opponent  points
16    2025_02_WAS_GB     2  WAS       GB    18.0
17   2025_02_CLE_BAL     2  CLE      BAL    17.0
18   2025_02_JAX_CIN     2  JAX      CIN    27.0
19   2025_02_NYG_DAL     2  NYG      DAL    37.0
20   2025_02_CHI_DET     2  CHI      DET    21.0
..               ...   ...  ...      ...     ...
267  2025_18_DAL_NYG    18  NYG      DAL     NaN
268  2025_18_WAS_PHI    18  PHI      WAS     NaN
269  2025_18_BAL_PIT    18  PIT      BAL     NaN
270   2025_18_SEA_SF    18   SF      SEA     NaN
271   2025_18_CAR_TB    18   TB      CAR     NaN

[512 rows x 5 columns]

In [43]:
result = schedule_total.merge(offensive_stats, on=['week', 'team'], how='left')
result = result.merge(defensive_stats, left_on=['week', 'opponent'], right_on=['week','opponent_team'], how='left')

In [44]:
result['team_top'] = result['team_x'].map(top_dict)
result['opp_top'] = result['opponent'].map(top_dict)

In [45]:
result

game_id  week team_x opponent  points opponent_team_x  \
0     2025_02_WAS_GB     2    WAS       GB    18.0              LV   
1    2025_02_CLE_BAL     2    CLE      BAL    17.0              GB   
2    2025_02_JAX_CIN     2    JAX      CIN    27.0             HOU   
3    2025_02_NYG_DAL     2    NYG      DAL    37.0              KC   
4    2025_02_CHI_DET     2    CHI      DET    21.0             DAL   
..               ...   ...    ...      ...     ...             ...   
507  2025_18_DAL_NYG    18    NYG      DAL     NaN             NaN   
508  2025_18_WAS_PHI    18    PHI      WAS     NaN             NaN   
509  2025_18_BAL_PIT    18    PIT      BAL     NaN             NaN   
510   2025_18_SEA_SF    18     SF      SEA     NaN             NaN   
511   2025_18_CAR_TB    18     TB      CAR     NaN             NaN   

     avg_passing_epa  avg_passing_cpoe  avg_rushing_epa  avg_receiving_epa  \
0           0.996192          2.028439         3.076621           7.561061   
1         -11.289482         -0.420814         1.091650          -2.759460   
2          -1.075935         -8.688481         2.338865           1.028223   
3          -3.046307         -1.656104        -2.214144           3.262790   
4           3.640705         -2.729754        -3.135422           7.202604   
..               ...               ...              ...                ...   
507              NaN               NaN              NaN                NaN   
508              NaN               NaN              NaN                NaN   
509              NaN               NaN              NaN                NaN   
510              NaN               NaN              NaN                NaN   
511              NaN               NaN              NaN                NaN   

     ...  avg_def_sack_yards  avg_def_qb_hits  avg_def_interceptions  \
0    ...           26.666667         6.333333               0.333333   
1    ...           28.000000         4.333333               0.333333   
2    ...           20.333333         5.666667               0.666667   
3    ...            8.666667         5.333333               1.333333   
4    ...            5.333333         6.000000               0.333333   
..   ...                 ...              ...                    ...   
507  ...                 NaN              NaN                    NaN   
508  ...                 NaN              NaN                    NaN   
509  ...                 NaN              NaN                    NaN   
510  ...                 NaN              NaN                    NaN   
511  ...                 NaN              NaN                    NaN   

     avg_def_interception_yards  avg_def_pass_defended  avg_def_tds  \
0                      8.333333               4.000000     0.000000   
1                      0.000000               2.666667     0.000000   
2                     29.000000               3.000000     0.333333   
3                     24.666667               3.666667     0.333333   
4                     20.333333               3.333333     0.000000   
..                          ...                    ...          ...   
507                         NaN                    NaN          NaN   
508                         NaN                    NaN          NaN   
509                         NaN                    NaN          NaN   
510                         NaN                    NaN          NaN   
511                         NaN                    NaN          NaN   

     avg_def_fumbles  avg_def_safeties  team_top  opp_top  
0           0.000000               0.0     27:50    30:21  
1           0.000000               0.0     29:54    28:10  
2           0.333333               0.0     31:05    26:59  
3           0.000000               0.0     30:28    29:26  
4           0.000000               0.0     31:00    31:48  
..               ...               ...       ...      ...  
507              NaN               NaN     30:28    29:26  
508              NaN               Na

In [51]:
import os

try:
    # Works when running as a .py script
    base_dir = os.path.dirname(__file__)
except NameError:
    # Fallback for Jupyter, Python in Excel, or interactive environments
    base_dir = os.getcwd()

data_path = os.path.join(base_dir, '..', 'Data', 'NFLData.csv')

os.makedirs(os.path.dirname(data_path), exist_ok=True)
result.to_csv(data_path, index=False)